In [1]:
import numpy as np
import torch
import torch.nn as nn
import random
import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms, models
from torchvision.datasets.folder import default_loader
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torch.autograd import Variable
from torchsummary import summary

In [2]:
batch_size = 128

train_dir = "/home/dawlat.akaila/Documents/AI_LABS/PROJECT_FINAL/datasets/CDA_chest_xray/train"
val_dir = "/home/dawlat.akaila/Documents/AI_LABS/PROJECT_FINAL/datasets/CDA_chest_xray/val"
test_dir = "/home/dawlat.akaila/Documents/AI_LABS/PROJECT_FINAL/datasets/CDA_chest_xray/test"

transform = transforms.Compose(
                [
                    
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                ]
            )

train_data = ImageFolder(train_dir, transform=transform, loader=default_loader)
val_data = ImageFolder(val_dir, transform=transform, loader=default_loader)
test_data = ImageFolder(test_dir, transform=transform, loader=default_loader)

In [3]:
def START_seed():
    seed = 9
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [4]:
# load dataset
START_seed()
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, num_workers = 8)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True, num_workers = 8)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, num_workers = 8)

In [5]:
# Extract features (X) and labels (y) for training, validation, and test sets
X_train, y_train = zip(*[(data, target) for data, target in train_data])
X_val, y_val = zip(*[(data, target) for data, target in val_data])
X_test, y_test = zip(*[(data, target) for data, target in test_data])

In [6]:
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [7]:
# Load the ResNet18 model
resnet18 = models.resnet18()

# Send 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
resnet18 = resnet18.to(device=device) 

# Loss and optimizer
lr=1e-4
n_epochs = 20  
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet18.parameters(), lr= lr) 

# summary(resnet18, (3, 224, 224))

In [8]:
resnet18.train()

# Training 
for epoch in range(n_epochs):
    train_loss = 0.0
    
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = resnet18(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))
    
# Save the trained model if needed
torch.save(resnet18.state_dict(), 'resnet18.pth')

Epoch: 1 	Training Loss: 2.336126
Epoch: 2 	Training Loss: 0.236519
Epoch: 3 	Training Loss: 0.106437
Epoch: 4 	Training Loss: 0.078365
Epoch: 5 	Training Loss: 0.045038
Epoch: 6 	Training Loss: 0.031339
Epoch: 7 	Training Loss: 0.013523
Epoch: 8 	Training Loss: 0.007419
Epoch: 9 	Training Loss: 0.008209
Epoch: 10 	Training Loss: 0.009104
Epoch: 11 	Training Loss: 0.008655
Epoch: 12 	Training Loss: 0.005379
Epoch: 13 	Training Loss: 0.002426
Epoch: 14 	Training Loss: 0.001775
Epoch: 15 	Training Loss: 0.002196
Epoch: 16 	Training Loss: 0.005918
Epoch: 17 	Training Loss: 0.008917
Epoch: 18 	Training Loss: 0.020022
Epoch: 19 	Training Loss: 0.016302
Epoch: 20 	Training Loss: 0.011587


In [9]:
# Load the weights
saved_weights_path = './resnet18.pth'
with torch.no_grad():
    resnet18.load_state_dict(torch.load(saved_weights_path))

# Remove the fully connected layers (classification layer)
resnet18 = nn.Sequential(*list(resnet18.children())[:-1]).to(device)

# Set the model to evaluation mode
resnet18.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [10]:
# Function to extract features from an image
def extract_features(image_tensor):
    image_tensor = image_tensor.to(device)
    with torch.no_grad():
        features = resnet18(image_tensor.unsqueeze(0))
        
        # Move the tensor to the CPU before converting to NumPy
        features_cpu = features.cpu()
    return features_cpu.flatten().numpy()

In [11]:
# Apply the feature extraction to each image in X_train
X_train_features = []

for i in range(len(X_train)):
    img_tensor = X_train[i]
    features = extract_features(img_tensor)
    X_train_features.append(features)

# Convert the list of features to a NumPy array
X_train_features = np.array(X_train_features)

print(X_train_features.shape)

(5216, 512)


In [12]:
# Apply the feature extraction to each image in X_train
X_val_features = []

for i in range(len(X_val)):
    img_tensor = X_train[i]
    features = extract_features(img_tensor)
    X_val_features.append(features)

# Convert the list of features to a NumPy array
X_val_features = np.array(X_val_features)

In [13]:
# Apply the feature extraction to each image in X_train
X_test_features = []

for i in range(len(X_test)):
    img_tensor = X_train[i]
    features = extract_features(img_tensor)
    X_test_features.append(features)

# Convert the list of features to a NumPy array
X_test_features = np.array(X_test_features)

In [14]:

np.save('/home/dawlat.akaila/Documents/AI_LABS/PROJECT_FINAL/denoising-pneumonia/Feature_Extraction/CDA/X_train_features.npy', X_train_features)
np.save('/home/dawlat.akaila/Documents/AI_LABS/PROJECT_FINAL/denoising-pneumonia/Feature_Extraction/CDA/X_val_features.npy', X_val_features)
np.save('/home/dawlat.akaila/Documents/AI_LABS/PROJECT_FINAL/denoising-pneumonia/Feature_Extraction/CDA/X_test_features.npy', X_test_features)

np.save('/home/dawlat.akaila/Documents/AI_LABS/PROJECT_FINAL/denoising-pneumonia/Feature_Extraction/CDA/y_train.npy', y_train)
np.save('/home/dawlat.akaila/Documents/AI_LABS/PROJECT_FINAL/denoising-pneumonia/Feature_Extraction/CDA/y_val.npy', y_val)
np.save('/home/dawlat.akaila/Documents/AI_LABS/PROJECT_FINAL/denoising-pneumonia/Feature_Extraction/CDA/y_test.npy', y_test)